### 4. Glove word vectors

[Pennington et. al., 2014. Glove:Global vectors gor words representation](https://nlp.stanford.edu/pubs/glove.pdf)

GloVe使用了词与词之间的共现（co-occurrence）信息。我们定义X为共现词频矩阵，其中元素$x_{ij}$为目标词j出现在词i的环境（context）的次数（等同于前面讲的$x_{ct}$）。这里的“环境”有多种可能的定义。举个例子，在一段文本序列中，如果词j出现在词i左边或者右边不超过10个词的距离，我们可以认为词j出现在词i的环境一次。令$x_i=\sum_kx_{ik}$为任意词出现在词i的环境的次数。那么，$$P_{ij}=P(j|i)=\frac{x_{ij}}{x_i}$$为词j出现在词i的环境的概率。这一概率也称为词i和词j的共现概率。

#### 4.1　Glove 使用词向量表达共现概率比值　

所谓共现概率比值，举个栗子：[Glove论文](https://nlp.stanford.edu/pubs/glove.pdf)中展示的例子

$ℙ(k∣ice): 0.00019 (k= solid) , 0.000066（k= gas），0.003（k= water），0.000017（k= fashion）$

$ℙ(k∣steam)：0.000022（k= solid），0.00078（k= gas），0.0022（k= water），0.000018（k= fashion）$

$\dfrac{ℙ(k∣ice)}{ℙ(k∣steam)}：8.9（k= solid），0.085（k= gas），1.36（k= water），0.96（k= fashion）$

- 与ice相关，而与steam不相关的词k，k=solid. $\dfrac{P_{ik}}{P_{jk}}=8.9$
- 与ice不相关而与steam相关的词k，k=gas.$\dfrac{P_{ik}}{P_{jk}}=0.085$
- 对于与ice和steam都相关的词k，k=water. $\dfrac{P_{ik}}{P_{jk}}= 1.36$
- 对于与ice和steam都不相关的词k，k=fashion. $\dfrac{P_{ik}}{P_{jk}}= 0.96$

由此可见，共现概率比值能比较直观地表达词之间的关系。

#### So，怎么用词向量来表达共现概率比值呢？
显然需要三个词向量，$v_k=\tilde v_k,不过用后者来表示其作为背景词～$
$$f(v_i,v_j,\tilde v_k) = \dfrac{P_{ik}}{P_{jk}}$$
函数f可能的设计并不唯一。可用向量之差：
$$f(v_i-v_j,\tilde v_k) = \dfrac{P_{ik}}{P_{jk}}$$
可以使用向量之间的内积把函数f的自变量进一步改写。
$$f((v_i-v_j)^T\tilde v_k) = \dfrac{P_{ik}}{P_{jk}}$$

由于任意一对词共现的对称性，我们希望以下两个性质可以同时被满足：
- 任意词作为中心词和背景词的词向量应该相等：对任意词i，$vi=\tilde v_i$
- 词与词之间共现次数矩阵X应该对称：对任意词i和j，$x_{ij}=x_{ji}$

实际上就是三个向量$v_i,v_j,v_k(=\tilde v_k)$之间的关系～怎么表示两个向量之间的关系呢？内积是一个很好的方法～～然后将内积转换为概率
$$exp(v_i^T\tilde v_k) = P_{ik} = \dfrac{x_{ik}}{x_i}\quad $$
**关键是这一步，很难想上去** $f(x) = e^x$
$$f((v_i-v_j)^T\tilde v_k) = \dfrac{f(v_i^T\tilde v_k)}{f(v_j^T\tilde v_k)}=\dfrac{P_{ik}}{P_{jk}}$$
这一步其实也就表明，我们要通过语料库学习得到怎样的词向量呢？就是这样的！两个词的内积的指数，等于他们的共现概率。

由此，我们推导出如何使用词向量来表达共现概率比值。

由上式可知，对于**共现概率＝向量内积的指数形式**，则

$$e^{v_i^T\tilde v_k} = \dfrac{x_{ik}}{x_i}$$

$$v_i^T\tilde v_k = log(x_{ik})-log(x_i)$$

把$log(x_i)$替换成两个偏移项之和$b_i+b_k$,
即可得到：
$$v_i^T\tilde v_k = log(x_{ik})-b_i-b_k$$　**这一步偏移项不太懂？？**

综上可知：对于三个词向量表达共现概率的比值，可以等价于表达两个词共现的词频的对数：
$$v_i^T\tilde v_k+b_i+b_k = log(x_{ik})$$

#### 4.2 如何学习得到词向量：损失函数

上式中词向量和偏移项都是要学习得到的。**我们希望左边和右边尽可能接近,并不是完全相等，因为偏移项是替换了$log(x_i)$的～右边的共现词频是通过训练数据统计得到的。** **这样就可以训练模型了～～**

给定词典大小Ｖ和权重函数$f(x_{ij})$，我们定义损失函数：
$$\sum^V_{i,j=1}f(x_{ij})(v_i^T\tilde v_j+b_i+b_j-log(x_{ij}))^2$$


对于权重函数f(x)(weight term)：
- if $x_{ij}=0$  $f(x_{ij}=0)$  共现词频至少大于１


- 当x<c(例如c=100),令$f(x)=(x/c)^{\alpha}(例如\alpha=0.75)。反之，令f(x)=1$

    有点类似于负采样中的方法，即不让词频特别高的the, of, a等的权重过大，也不让durian这样的权重太小。

需要注意的是，损失函数的计算复杂度与共现词频矩阵X中非零元素的数目呈线性关系。我们可以从X中随机采样小批量非零元素，使用随机梯度下降迭代词向量和偏移项。

需要注意的是，对于任意一对i,j，损失函数中存在以下两项之和

$$f(x_{ij})(v_i^T\tilde v_i+b_i+b_j-log(x_{ij}))^2+f(x_{ji})(v_j^T\tilde v_i+b_j+b_i-log(x_{ji}))^2$$

由于xij=xji，对调v和ṽ 并不改变损失函数中这两项之和的值。也就是说，在损失函数所有项上对调v和ṽ 也不改变整个损失函数的值。因此，任意词的中心词向量和背景词向量是等价的。只是由于初始化值的不同，同一个词最终学习到的两组词向量可能不同。当所有词向量学习得到后，GloVe使用一个词的中心词向量与背景词向量之和作为该词的最终词向量。